## <font color='seagreen'> Segmentation2 2D Segment Single Macrophages </font>

In this Notebook, we will perform the segmenation of single macrophages within Napari in 2D. The segmenation was performed manually but aided by some widgets shown in this notebook.

In [ ]:
#Load libraries.
import napari
import os
import numpy as np
from glob import glob
from joblib import load
from functools import partial
from skimage import io,segmentation,data
from magicgui import magicgui
from pathlib import Path

***

### Defining Parameters

Define parameters to select images to analyse and correctly display these in Napari:

In [ ]:
hours_options=[0,12,24,36,48,72]
hour=hours_options[0]
#image path
path_imgs=fr'..\Data\{hour} Hr\{hour} hr\*.TIF'
path_DICs=fr'..\Data\{hour} Hr\DIC\*.TIF'
#select fov to observe; which image in folder
fov=3
#shape of your images (c,z,x,y)
shape=(3,41,2304,2304)
#which layers are in focus?
focus=[15,35]
#channel order
channels=['CY3_5','DAPI','CY5']
#colors to display different channels in Napari
colors=['green','blue','bop purple']
#set intensity ranges (these can be adjusted in napari as well).
cls=[[0.05,1],[0,8,1],[0.05,1]]

Parameters for feature stack establishment. No need to change to anything.

In [ ]:
sigma_min = 1
sigma_max = 20

features_func = partial(feature.multiscale_basic_features,
                        intensity=True, edges=False, texture=True,
                        sigma_min=sigma_min, sigma_max=sigma_max)

Load image:

In [ ]:
filenames=glob(path_imgs)
img=io.imread(filenames[fov], plugin='pil')
img=np.reshape(img,shape)
DIC_filenames=glob(path_DICs)
DIC=io.imread(DIC_filenames[fov])
print(img.shape,DIC.shape)

Zproject the image:

In [ ]:
zproject=np.amax(img[:,focus[0]:focus[1],...],axis=1,keepdims=True)
zproject.shape

select DAPI threshold. You can also select this threshold later in Napari. If you want to do this set threshold to None.

In [ ]:
#select DAPI threshold 
DAPI_threshold=None

### Segmentation

Load images in Napari and create cell and nuclei masks. Additionally, we can annotate mother bud relationship:

In [ ]:
viewer = napari.Viewer()

for i in range(0,shape[0]):
    x=zproject[i,...]
    x=np.squeeze(x,axis=0)
    viewer.add_image(x,name=channels[i],blending='additive',colormap=colors[i])
viewer.add_image(DIC,opacity=0.25)

from napari.layers import Labels
from napari.types import LabelsData
from skimage.morphology import remove_small_holes
from skimage.morphology import remove_small_objects

@magicgui(auto_call=False,call_button='Remove small labels')
def remove(labels: Labels, area: int = 100) -> LabelsData:
    rlabels=remove_small_objects(labels.data,area)
    print(labels.name)
    try:
        viewer.layers.remove(labels.name)
    except:
        pass
    viewer.add_labels(rlabels,name=labels.name)
    
viewer.window.add_dock_widget(remove)

Save single-cell Macrophages masks.

In [ ]:
root,filename=os.path.split(filenames[fov])

# save cell_masks:
cell_spath=os.path.join(root,"cell masks")
fcell_spath=os.path.join(cell_spath,filename)
Path(cell_spath).mkdir(parents=True, exist_ok=True)

#uncomment to overwrite and save created mask
#io.imsave(fcell_spath,viewer.layers['Labels'].data.astype(np.uint16))

***

In the next Notebook (**Notebooks_Data_exploration\Notebook0_Extract_data_from_Masks_and_Fluorescent_images.ipynb**), we will extract the fluorescent intensities for each individual cell mask label of both macrophages and bacteria.